# qBraid-sdk Circuits demo

The following notebook provides a quick demo of the qBraid circuit implementation. The overall goal of this notebook is to provide a quick intuitive intro for qBraid-sdk engineers/researchers who will use the qBraid-sdk for development of the VQE as well as for research purposes.

In [1]:
import qbraid
from qbraid.circuits.circuit import Circuit
from qbraid.circuits.drawer import drawer

## Initializing a circuit

Create a circuit with 3 qubits. All circuits require the number of qubits (channels) that will be used.
Optional arguments are naming the circuit as well as changing the update rule. A `Circuit` is a collection of `Moment`s.

In [2]:
circuit = Circuit(num_qubits=3,name="test_circuit")

In [3]:
print(circuit)

Circuit(test_circuit, 3 qubits, 0 gates)


In [4]:
from qbraid.circuits import UpdateRule

In [5]:
another_circuit = Circuit(3,"another_circuit",UpdateRule.NEW)

## How to add gates to a circuit
Let's add some gates to a circuit. To do so we need to import the `Instruction` module which allows us to define the qubits that the gate will operate on.An `Instruction` is an effect that operates on a specific subset of `Qubits`, the most common type of `Instruction` is a `Gate`. A gate will not be appended if the qubit is not specified using the Instruction module.  Each `Instruction` must have a disjoint set of qubits from the other `Instruction` in the `Moment`. A `Moment` can be thought of as a vertical slice of a quantum circuit diagram.

In [6]:
from qbraid.circuits.instruction import Instruction

### Single qubit gate
We'll begin by creating a Hadamard, Pauli X, and Y gates.

In [7]:
from qbraid.circuits import H,X,Y
h =  Instruction(H(),qubits=0) # Hadamard which acts on the 0-th qubit.
x =  Instruction(X(),qubits=[0]) # Pauli X which acts on the 0-th qubit.
y =  Instruction(Y(),qubits=1) # Pauli Y which acts on the 1 qubit.

Note that all qubits are put into a list.

In [8]:
print(h)
print(x)

Instruction ([0] qubits, <qbraid.circuits.library.standard_gates.h.H object at 0x7f8c014f1a30> gate)
Instruction ([0] qubits, <qbraid.circuits.library.standard_gates.x.X object at 0x7f8c014f1a90> gate)


Appending the instructions to the circuit results in the creation of moments. A `Moment` is a collection of `Instructions` that all act during the same abstract time slice. Printing out the moments shows that the circuit has two `Moment` objects where Hadamard acts in the first moment and since the X gate act

In [9]:
circuit = Circuit(num_qubits=3,name="test_circuit")
circuit.append([h,x,y])
print(circuit.moments)
drawer(circuit)

True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1af0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>, <qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>]")]
Qubit 0:|┤H├|┤X├
Qubit 1:|---|┤Y├
Qubit 2:|---|---


### Two qubit gates

The story is similar for two qubit gates. The only difference is that two qubits must be specified when "wrapping" the gate with the `Instruction`.

In [10]:
from qbraid.circuits import CH,CX,CY
ch =  Instruction(CH(),qubits=[0,2]) # Control Hadamard which acts on the 0-th qubit.
cx =  Instruction(CX(),qubits=[0,1]) # Control Pauli X which acts on the 0-th qubit.
cy =  Instruction(CY(),qubits=[2,1]) # Control Pauli Y which acts on the 1 qubit.

In [11]:
print(ch)
print(cx)

Instruction ([0, 2] qubits, <qbraid.circuits.library.standard_gates.h.CH object at 0x7f8c014f19a0> gate)
Instruction ([0, 1] qubits, <qbraid.circuits.library.standard_gates.x.CX object at 0x7f8c014f1cd0> gate)


Since the `CH` and `CX` gates both require the 0-th qubit for the operation, a new `Moment` is created. Appending the three gates results in three `Moment`s.

In [12]:
circuit = Circuit(num_qubits=3,name="test_circuit")
circuit.append([ch,cx,cy])
print(circuit.moments)
drawer(circuit)

True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f16d0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1d90>]")]
Qubit 0:|┤CH├|┤CX├|-----
Qubit 1:|----|┤CX├|┤CSY├
Qubit 2:|┤CH├|----|┤CSY├


## Update Rules

The circuit will by default update according to the `NEW_THEN_INLINE` rule. `UpdateRule` defines how `Instructions `are placed in a `Circuit` when requested to be inserted at a given location. Here, a location is identified by the index of the `Moment` (in the `Circuit`) where the insertion is requested to be placed at (in the case of `Circuit.append`, this means inserting at the `Moment`, at an index one greater than the maximum moment index in the `Circuit`). There are four such rules: `UpdateRule.EARLIEST`, `UpdateRule.NEW`, `UpdateRule.INLINE` and `UpdateRule.NEW_THEN_INLINE`. See [Circ Insert Strategy](https://quantumai.google/reference/python/cirq/circuits/InsertStrategy#NEW_THEN_INLINE) for more information.

### `UpdateRule.EARLIEST`

Scans backward from the insert location until a moment with operations touching qubits affected by the operation to insert is found. The operation is added to the moment just after that location.

We can tell from the below example that since `CH` acts on the 0 and 2 qubit and `Y` operates on the 1 qubit they are placed in the same `Moment`. The next gate,`X` however operates on the 0 qubit. Therfore, a new `Moment` is created. Since the second `X` gate operates on the same qubit as the first `X` a new `Moment` is once again created. 

In [13]:
circuit = Circuit(num_qubits=3, name="test_circuit",update_rule=UpdateRule.EARLIEST)
circuit.append(ch)
circuit.append([y,x])
circuit.append(x)
print(circuit.moments)
drawer(circuit)

True
True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>, <qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>]")]
Qubit 0:|┤CH├|┤X├|┤X├
Qubit 1:|┤Y├-|---|---
Qubit 2:|┤CH├|---|---


### `UpdateRule.NEW`

Every operation that is inserted is created in a new moment.

We see that there are three `Moment` objects in the list of moments confirming that a new moment is created for
every operation.

In [14]:
circuit = Circuit(num_qubits=3, name="test_circuit",update_rule=UpdateRule.NEW)
circuit.append(h)
circuit.append(x)
circuit.append(y)
print(circuit.moments)
drawer(circuit)

True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1af0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>]")]
Qubit 0:|┤H├|┤X├|---
Qubit 1:|---|---|┤Y├
Qubit 2:|---|---|---


### `UpdateRule.INLINE`

Attempts to add the operation to insert into the moment just before the desired insert location. But, if there’s already an existing operation affecting any of the qubits touched by the operation to insert, a new moment is created instead.

The circuit appends the `CH` gate (operateson the 0 and 2 qubit) in its own `Moment` because the `CY` gate also operates on the 2 qubit. The `H` gate however is appended into the same `Moment` as the `CY` gate since the `H` gate operates on the 0 qubit where `CY` operates on the 1 and 2 qubit. The `Y` gate acts on the 1 qubit and `X` gate acts on the 0 qubit so they are added into the same `Moment`. 

In [15]:
circuit = Circuit(num_qubits=3, name="test_circuit",update_rule=UpdateRule.INLINE)
circuit.append(ch)
circuit.append(cy)
circuit.append([h,y,x])
print(circuit.moments)
drawer(circuit)

True
True
True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1d90>, <qbraid.circuits.instruction.Instruction object at 0x7f8c014f1af0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>, <qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>]")]
Qubit 0:|┤CH├|┤H├--|┤X├
Qubit 1:|----|┤CSY├|┤Y├
Qubit 2:|┤CH├|┤CSY├|---


### `UpdateRule.NEW_THEN_INLINE`

Creates a new moment at the desired insert location for the first operation, but then switches to inserting operations inline.

The first append creates a single moment with an `CH` on the first qubit. Then, the append with the `InsertStrategy.NEW_THEN_INLINE` strategy begins by inserting the `Y` in a new `Moment` (the `InsertStrategy.NEW` in `InsertStrategy.NEW_THEN_INLINE`). Subsequent appending is done `InsertStrategy.INLINE`, so the next `CH` on the first qubit is appending in the just created `Moment`.

In [16]:
circuit = Circuit(num_qubits=3, name="test_circuit")
circuit.append(ch)
circuit.append([y,ch])
print(circuit.moments)
drawer(circuit)

True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>, <qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>]")]
Qubit 0:|┤CH├|┤CH├
Qubit 1:|----|┤Y├-
Qubit 2:|┤CH├|┤CH├


### Different Update Rules
We can also use different update rules by specifying during appending.

In [17]:
circuit = Circuit(num_qubits=3, name="test_circuit")
circuit.append(ch,update_rule=UpdateRule.INLINE)
circuit.append(cy,update_rule=UpdateRule.EARLIEST)
circuit.append([h,y,x],update_rule=UpdateRule.NEW)
print(circuit.moments)
drawer(circuit)

True
True
True
True
True
[Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f13a0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1d90>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1af0>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1c70>]"), Moment("[<qbraid.circuits.instruction.Instruction object at 0x7f8c014f1bb0>]")]
Qubit 0:|┤CH├|-----|┤H├|---|┤X├
Qubit 1:|----|┤CSY├|---|┤Y├|---
Qubit 2:|┤CH├|┤CSY├|---|---|---


## Deutsch Algorithm Example
The very first indication that quantum computers could be more powerful than classical computers was provided by David Deutsch in his 1985 paper. This is a modified implementation of the Cirq Deutsch algorithm see [Deutsch-Jozsa](https://quantumai.google/cirq/tutorials/educators/intro#the_deutsch-jozsa_algorithm) for more information.

This algorithm was extended by Deutsch and Richard Jozsa to a more convincing algorithmic seperation and what is now called the Deutsch-Jozsa algorithm.  In this section we will show how to write circuits for the Deutsch algorithm.

Let's begin with the Deutsch algorithm.  In Deutsch's algorithm you are given access to a box which computes a one bit boolean function.  That is it is a box which takes in a bit and outputs a bit.  If we want to be a mathematician or theoretical computer scientist we write the function $f$ as $f: \{0, 1\} \rightarrow \{0, 1\}$.  There are exactly four such boolean functions which we can write out in a table

| $x$ | $f_0$ | $f_1$ | $f_x$ | $f_{\bar{x}}$ |
| --- |  --- | --- | --- | --- |
| 0 | 0 | 1 | 0 | 1
| 1 | 0 | 1 | 1 | 0

The first two of these are *constant* functions, $f_0$ and $f_1$.  That is they always output a constant value (independent of the input).  The other two $f_x$ and $f_\bar{x}$ are *balanced*.  Over their inputs $0$ and $1$, they have an equal number of $0$s and $1$s in their truth table.  

We can now state Deutsch's problem:

> Given access to a one bit input one bit output boolean function, determine by querying the function whether the function is *balanced* or *constant*.

It shouldn't take you much to convince yourself that in order to solve this problem classically you need to call the function on both possible input values.  The easiest way to see this is just to consider what happens if you query the function on one particular input and notice that, for either input, learning the value of the function does not separate the constant from balanced functions. In summary:

*Classically one must query the binary function twice to distinguish the constant function from the balanced function.*


Now lets turn to the quantum approach to this problem.  There is one bit of book keeping we need to take care of.  Above we have described a classical function on bits that is not reversible.  That is, knowing the values of the output does not allow us to determine uniquely the value of the input.  In order to run this on a quantum computer, however we need to make this computation reversible.  A trick for taking a classical non-reversible function and making it "quantum happy" is to compute the value in an extra register and store the input.  Suppose we have an $n$ bit input $x$ and we are computing a (potentially non-reverisble) boolean function $f(x)$.  Then we can implement this via a Unitary $U_f$ that acts like on $n + 1$ qubits

$$
U_f |x\rangle |y\rangle = |x\rangle | y \oplus f(x)\rangle .
$$

Here $\oplus$ is addition modulo $2$ (XOR) and we have identified how $U_f$ acts by its action on all computational basis states $|x\rangle$ ($n$ input qubits) and $|y\rangle$ ($1$ output qubit). To see that this is reversible one can note that applying the transformation twice returns the state to its original form.

Let's see how to implement these functions in qBraid.

$f_0$ enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |00\rangle \\
|01\rangle &\rightarrow&  |01\rangle \\
|10\rangle &\rightarrow&  |10\rangle \\
|11\rangle &\rightarrow&  |11\rangle \\
\end{eqnarray}
$$
Well this is just the identity transform, i.e. an empty circuit.

$f_1$ enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |01\rangle \\
|01\rangle &\rightarrow&  |00\rangle \\
|10\rangle &\rightarrow&  |11\rangle \\
|11\rangle &\rightarrow&  |10\rangle \\
\end{eqnarray}
$$
This is the `qbraid.circuits.X` bit flip gate on the second qubit.

$f_x$ enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |00\rangle \\
|01\rangle &\rightarrow&  |01\rangle \\
|10\rangle &\rightarrow&  |11\rangle \\
|11\rangle &\rightarrow&  |10\rangle \\
\end{eqnarray}
$$
This is nothing more than a `qbraid.circuits.CX` from the first bit to the second bit.

Finally $f_\bar{x}$ enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |01\rangle \\
|01\rangle &\rightarrow&  |00\rangle \\
|10\rangle &\rightarrow&  |10\rangle \\
|11\rangle &\rightarrow&  |11\rangle \\
\end{eqnarray}
$$
which is a `qbraid.circuits.CX` from the first bit to the second bit followed by a `qbraid.circuits.X` on the second bit.

We can encapulate these functions into a dictionary from a oracle name to the operations in the circuit needed to enact this function.

### Deutsch's Algorithm

In [18]:
"""Store the operations to query each function in a dictionary."""
#  Initialize.
deutsch_circuit = Circuit(num_qubits=2,name="deutsch_circuit")

# Define the dictionary of operations. The key of each dictionary entry
# is the subscript of the function f in the above explanatory text.
oracles = {
    '0': [],
    '1': [Instruction(qbraid.circuits.X(),1)],
    'x': [Instruction(qbraid.circuits.CX(),[0,1])],
    'notx': [Instruction(qbraid.circuits.CX(),[0,1]), Instruction(qbraid.circuits.X(),1)]
}    

We now turn to Deutsch's algorithm.  Suppose we are given access to the reversible oracle functions we have defined above.  By a similar argument for our irreversible classical functions you can show that you cannot distinguish the balanced from the constant functions by using this oracle only once.  But now we can ask the question: what if we are allowed to query this box in superposition, i.e. what if we can use the power of quantum computing?

Deutsch was able to show that you could solve this problem now, with quantum computers, using only a single query.  To see how this works we need two simple insights.

Suppose that we prepare the second qubit in the superposition state $|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$ and apply the oracle.  Then we can check that
$$ 
U_f |x\rangle |-\rangle = U_f|x\rangle \frac{1}{\sqrt{2}}(|0\rangle -|1\rangle ) = |x\rangle \frac{1}{\sqrt{2}}(|f(x)\rangle -|f(x) \oplus 1\rangle ) =  (-1)^{f(x)} |x\rangle |-\rangle .
$$  
This is the so called "phase kickback trick".  By applying $U_f$ onto a target which is in superposition, the value of the function ends up showing up in the global phase.  

How can we leverage this to distinguish between the constant and balanced functions?  Note that for the constant functions the phase that is applied is the same for all inputs $|x\rangle$, whereas for the balanced functions the phase is different for each value of $x$.  In other words, if we use the phase kickback trick then for each of the oracles we apply the following transform on the first qubit:

$$
\begin{eqnarray}
f_0 \rightarrow I, &&
f_1 \rightarrow -I, &&
f_x \rightarrow Z, &&
f_\bar{x} \rightarrow -Z &&
\end{eqnarray}
$$

Now we only need, on the first qubit, to distinguish between the identity gate and the $Z$ gate.  But we can do this by recalling the identity

$$ 
H Z H = X
$$

where $H$ is the Hamadard gate.

This means that we can turn a phase flip into a bit flip by applying Hadamards before and after the phase flip.  If we look at the constant and balanced functions we see that this means that the constant functions will be proportional to $I$ and the balanced functions will be proportional to $X$.  If we feed in $|0\rangle$ to this register, then in the first cases we will only see $|0\rangle$ and in the second case we will only see $|1\rangle$.  In other words we will be able to distinguish constant from balanced using a single query of the oracle.

Let's code this up.

In [19]:

"""Creating the circuit used in Deutsch's algorithm."""
def deutsch_algorithm(oracle,circuit):
    """Returns the circuit for Deutsch's algorithm given an input
    oracle, i.e., a sequence of operations to query a particular function.
    """
    circuit.append([Instruction(qbraid.circuits.X(),1),Instruction(qbraid.circuits.H(),0), Instruction(qbraid.circuits.H(),1),*oracle])
    circuit.append([Instruction(qbraid.circuits.H(),0)],update_rule=UpdateRule.INLINE)
    circuit.append(Instruction(qbraid.circuits.Measure(),0)) #not implemented
    return circuit.moments
for key, oracle in oracles.items():
    deutsch_circuit = Circuit(num_qubits=2,name="deutsch_circuit")
    print(f"Circuit for f_{key}:")
    deutsch_algorithm(oracle,deutsch_circuit)
    drawer(deutsch_circuit)

Circuit for f_0:
True
True
True
True
True
Qubit 0:|┤H├|┤H├|┤measure├
Qubit 1:|┤X├|┤H├|---------
Circuit for f_1:
True
True
True


CircuitError: "Operation of type <class 'qbraid.circuits.instruction.Instruction'> not appendable in circuit of 2 qubits"

We interpret the simulation results as follows:

- For the first two functions $f_0$ and $f_1$, we always measure $0$. Therefore, we know that these functions are constant.
- For the second two functions $f_x$ and $f_{\bar{x}}$, we always measure $1$. Therefore, we know that these functions are balanced.

David Deutsch, "Quantum Theory, the Church-Turing Principle and the Universal Quantum Computer" Proc. R. Soc. Lond. A 400 97–117. http://doi.org/10.1098/rspa.1985.0070